In [2]:
import openai
import pandas as pd

In [3]:
client = openai.Client()

In [4]:
file = client.files.create(
    file=open("sales_data.csv", "rb"),
    purpose="assistants"
)

In [5]:
print(file.id)

file-4yaR3vJvEzHNG1Vi77eo1c


In [7]:
assistant = client.beta.assistants.create(
    name="Analista de dados",
    instructions="voce é um analista que analisa dados sobre vendas",
    tools=[{"type":"code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
    model="gpt-4o"
)

In [17]:
# pergunta = "Qual rating medio das vendas do supermercado."
pergunta = "Gere um grafico pizza com o percentual de vendas por linha de produto."

In [18]:
 # criacao da thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

In [19]:
# executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

In [20]:
# aguarda a thread rodar
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [21]:
# Verifica a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")

SyncCursorPage[Message](data=[Message(id='msg_kwi0r8CqQBu9sAwH40bnq9e4', assistant_id='asst_OVun1Z8No9ycFKkiy8jF4IHK', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-MHUxrr4VvNtrBEZ9xwssuz', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico de pizza que mostra o percentual de vendas por linha de produto. Cada fatia representa a contribuição percentual de uma linha de produto para o total de vendas. Se precisar de mais alguma análise ou modificação no gráfico, é só me avisar!'), type='text')], created_at=1746574823, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_EV1xVyIEgViSvyb3AU8NjurF', status=None, thread_id='thread_KiWWlLDvwx523KvjyZQ4WArq'), Message(id='msg_B08uN1skujcY5YkPCx4AvN7N', assistant_id='asst_OVun1Z8No9ycFKkiy8jF4IHK', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=

In [22]:
# analizando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [23]:
mensagens.data[0].content[0].text.value

AttributeError: 'ImageFileContentBlock' object has no attribute 'text'

In [26]:
for step in run_steps.data[::-1]:
    print(f"\n=== Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("===========")
            print(tool_call.code_interpreter.input)
            print("===========")
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files/{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem {file_id} foi salva!")


=== Step message_creation
Primeiro, vamos examinar o conteúdo do arquivo que você enviou para entender sua estrutura e identificar a linha de produto e as vendas. Depois, geraremos o gráfico de pizza.

=== Step tool_calls
import pandas as pd

# Ler o arquivo para entender sua estrutura
file_path = '/mnt/data/file-4yaR3vJvEzHNG1Vi77eo1c'
data = pd.read_csv(file_path)

# Exibir as primeiras linhas do dataframe para entender sua estrutura
data.head()

=== Step message_creation
O conjunto de dados contém uma coluna chamada "Product line" que indica a linha de produto, e uma coluna "Total" que representa o total das vendas. Vamos usar essas informações para calcular o percentual de vendas por linha de produto e gerar um gráfico de pizza.

=== Step tool_calls
import matplotlib.pyplot as plt

# Calcular o valor total de vendas por linha de produto
sales_by_product_line = data.groupby('Product line')['Total'].sum()

# Calcular percentual de vendas por linha de produto
sales_percentage = (sale